In [1]:
import os

In [2]:
from osgeo import gdal

In [3]:
import zipfile
import tarfile

In [4]:
import subprocess

In [5]:
import glob
import shutil

In [6]:
indir = r'E:\Temp\nightlights'


In [7]:
infiles = glob.glob(indir+'\\*.tgz')

In [8]:
infiles

['E:\\Temp\\nightlights\\SVDNB_npp_20160501-20160531_00N060E_vcmslcfg_v10_c201606281430.tgz',
 'E:\\Temp\\nightlights\\SVDNB_npp_20160501-20160531_00N060W_vcmslcfg_v10_c201606281430.tgz',
 'E:\\Temp\\nightlights\\SVDNB_npp_20160501-20160531_00N180W_vcmslcfg_v10_c201606281430.tgz',
 'E:\\Temp\\nightlights\\SVDNB_npp_20160501-20160531_75N060E_vcmslcfg_v10_c201606281430.tgz',
 'E:\\Temp\\nightlights\\SVDNB_npp_20160501-20160531_75N060W_vcmslcfg_v10_c201606281430.tgz',
 'E:\\Temp\\nightlights\\SVDNB_npp_20160501-20160531_75N180W_vcmslcfg_v10_c201606281430.tgz',
 'E:\\Temp\\nightlights\\SVDNB_npp_20160601-20160630_00N060E_vcmslcfg_v10_c201608101833.tgz',
 'E:\\Temp\\nightlights\\SVDNB_npp_20160601-20160630_00N060W_vcmslcfg_v10_c201608101833.tgz',
 'E:\\Temp\\nightlights\\SVDNB_npp_20160601-20160630_00N180W_vcmslcfg_v10_c201608101833.tgz',
 'E:\\Temp\\nightlights\\SVDNB_npp_20160601-20160630_75N060E_vcmslcfg_v10_c201608101833.tgz',
 'E:\\Temp\\nightlights\\SVDNB_npp_20160601-20160630_75N060W

In [9]:
for f in infiles:
    base = os.path.basename(f)
    daterange = base.split('_')[2]
    destdir = os.path.join(indir,daterange)
    if not os.path.exists(destdir):
        os.mkdir(destdir)
    shutil.move(f, destdir)
    

In [10]:
indirs = [d for d in glob.glob(indir+"\\*") if os.path.isdir(d)]


In [11]:
from joblib import Parallel, delayed


In [12]:
sorted(indirs)

['E:\\Temp\\nightlights\\20160501-20160531',
 'E:\\Temp\\nightlights\\20160601-20160630',
 'E:\\Temp\\nightlights\\20160701-20160731',
 'E:\\Temp\\nightlights\\20160801-20160831',
 'E:\\Temp\\nightlights\\20160901-20160930']

In [16]:
vrtcommand = "gdalbuildvrt {0} {1}"
transcommand = "gdal_translate -of GTiff -co COMPRESS=LZW "+\
    "-co PREDICTOR=2 -co TILED=YES -co SPARSE_OK=TRUE -co BIGTIFF=YES "+\
    "--config GDAL_CACHEMAX 8000 {0} {1}"
ovcommand = "gdaladdo -ro --config COMPRESS_OVERVIEW LZW --config USE_RRD NO " +\
        "--config TILED YES {0} 2 4 8 16 32 64 128 256 --config GDAL_CACHEMAX 8000"
statcommand = "gdalinfo -stats {0} >nul"
def translateDir(dIn, dOut=None):
    if dOut is None:
        dOut = dIn
    print "Operating in folder "+dIn
    filetag = 'vcmslcfg' # or vcmcfg for stray-light-excluded
    tars = glob.glob(dIn+"\\"+"*"+filetag+"*.tgz")
    if len(tars)<6:
        print "incomplete folder!"
        return
    dirTileList = []
    subDirName = os.path.basename(dIn)
    outFile = os.path.join(dOut, "VIIRS.DNB.StrayLightCorrected."+subDirName)
    
    if os.path.exists(outFile + ".tif"):
        print "Already done! Skipping"
        return
    #print outFile + " doesn't exist: running!"
    #return
    for t in tars:
        if t.find(filetag) == -1:
            continue
        tar = tarfile.open(t)
        #tardir = os.path.dirname(t)
        members = tar.members
        for m in members:
            if m.name.find('avg_rad') != 1:
                if os.path.exists(os.path.join(dIn, m.name)):
                    print m.name +" exists - skipping extraction!"
                    continue
                print "Extracting "+m.name
                
                tar.extract(m, dIn)
                dirTileList.append(os.path.join(dIn, m.name))
    
    
    vrtBuilder = vrtcommand.format(outFile+".vrt", os.path.join(dIn,"*"+filetag+"*.tif"))
    print vrtBuilder
    subprocess.call(vrtBuilder)
        
    transBuilder = transcommand.format(outFile+".vrt", outFile+".tif")
    print transBuilder
    subprocess.call(transBuilder)
    
    ovBuilder = ovcommand.format(outFile+".tif")
    print ovBuilder
    subprocess.call(ovBuilder)
    
    statBuilder = statcommand.format(outFile+".tif")
    print statBuilder
    subprocess.call(statBuilder)
    
    for f in dirTileList:
        os.remove(f)

In [15]:
#Parallel(n_jobs=4)(delayed(translateDir)(d) for d in indirs)
outDir = r'\\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\Other_Global_Covariates\NightTimeLights\VIIRS_DNB_Monthly'
outDir = r'C:\Temp\nightlights'
for d in sorted(indirs[1:]):
    translateDir(d, outDir)

Operating in folder E:\Temp\nightlights\20160601-20160630
Extracting SVDNB_npp_20160601-20160630_00N060E_vcmslcfg_v10_c201608101833.avg_rade9.tif
Extracting SVDNB_npp_20160601-20160630_00N060W_vcmslcfg_v10_c201608101833.avg_rade9.tif
Extracting SVDNB_npp_20160601-20160630_00N180W_vcmslcfg_v10_c201608101833.avg_rade9.tif
Extracting SVDNB_npp_20160601-20160630_75N060E_vcmslcfg_v10_c201608101833.avg_rade9.tif
Extracting SVDNB_npp_20160601-20160630_75N060W_vcmslcfg_v10_c201608101833.avg_rade9.tif
Extracting SVDNB_npp_20160601-20160630_75N180W_vcmslcfg_v10_c201608101833.avg_rade9.tif
gdalbuildvrt C:\Temp\nightlights\VIIRS.DNB.StrayLightCorrected.20160601-20160630.vrt E:\Temp\nightlights\20160601-20160630\*vcmslcfg*.tif
gdal_translate -of GTiff -co COMPRESS=LZW -co PREDICTOR=2 -co TILED=YES -co SPARSE_OK=TRUE -co BIGTIFF=YES --config GDAL_CACHEMAX 8000 C:\Temp\nightlights\VIIRS.DNB.StrayLightCorrected.20160601-20160630.vrt C:\Temp\nightlights\VIIRS.DNB.StrayLightCorrected.20160601-20160630.t

In [14]:
sorted(indirs)[-1]


'\\\\129.67.26.176\\map_data\\hsg\\VIIRS\\downloads\\20151201-20151231'

In [15]:
translateDir(sorted(indirs)[-1])

Operating in folder \\129.67.26.176\map_data\hsg\VIIRS\downloads\20151201-20151231
Extracting SVDNB_npp_20151201-20151231_75N060W_vcmslcfg_v10_c201601251413.avg_rade9.tif
Extracting SVDNB_npp_20151201-20151231_75N060E_vcmslcfg_v10_c201601251413.avg_rade9.tif
Extracting SVDNB_npp_20151201-20151231_00N060W_vcmslcfg_v10_c201601251413.avg_rade9.tif
Extracting SVDNB_npp_20151201-20151231_00N180W_vcmslcfg_v10_c201601251413.avg_rade9.tif
Extracting SVDNB_npp_20151201-20151231_00N060E_vcmslcfg_v10_c201601251413.avg_rade9.tif
Extracting SVDNB_npp_20151201-20151231_75N180W_vcmslcfg_v10_c201601251413.avg_rade9.tif
gdalbuildvrt \\129.67.26.176\map_data\hsg\VIIRS\downloads\20151201-20151231\VIIRS.DNB.StrayLightCorrected.20151201-20151231.vrt \\129.67.26.176\map_data\hsg\VIIRS\downloads\20151201-20151231\*vcmslcfg*.tif
gdal_translate -of GTiff -co COMPRESS=LZW -co PREDICTOR=2 -co TILED=YES -co SPARSE_OK=TRUE -co BIGTIFF=YES --config GDAL_CACHEMAX 8000 \\129.67.26.176\map_data\hsg\VIIRS\downloads\201